In [1]:
import matplotlib
matplotlib.use('pdf')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
plt.style.use('default')

In [13]:
def human_format(x):
    sigpart = 0
    suffix = ""
    if x < 1000:
        return "{:.3g}".format(x)
    elif x < 1000000:
        sigpart = x / 1000
        suffix = "k"
    elif x < 1000000000:
        sigpart = x / 1000000
        suffix = "M"
    else:
        sigpart = x / 1000000000
        suffix = "B"
    return "{:.3g} {}".format(sigpart, suffix)
def get_algorithm(path):
    algs = ["rkde", "ic2", "sklearn", "nocut", "simple"]
    for alg in algs:
        if alg in path:
            return alg
def get_dataset(path):
    datasets = {
        "bgauss": "gauss",
        "hep": "hep",
        "tmy3_4": "tmy3_4",
        "tmy3.": "tmy3",
        "mnist_64": "mnist",
        "home": "home",
    }
    for dataset in datasets.keys():
        if dataset in path:
            return datasets[dataset]
markers = {
    "ic2": "o",
    "simple": "^",
    "sklearn": "s",
    "rkde": "d",
    "nocut": "8",
    "ks": "*",
}

In [4]:
def get_p(path):
    underscore_idx = path.index("_")
    dot_idx = path.index(".out")
    return int(path[underscore_idx+1:dot_idx])

In [8]:
df = pd.read_csv("../experiments/p_raiders/scale_p.csv")

In [9]:
df["r_algorithm"] = df["out_path"].map(get_algorithm)
df["r_dataset"] = df["out_path"].map(get_dataset)
df["throughput"] = (
    1000 * df["num_test"] / df["test_time"]
)
df["a_throughput"] = (
    1000 * df["num_train"] 
    / (df["test_time"]*df["num_train"]/df["num_test"] + df["train_time"])
)
df["p"] = df["out_path"].map(get_p)

In [10]:
df = df.sort_values(by=["r_algorithm", "p"])

In [11]:
algorder = ["ic2"]

In [20]:
fig = plt.figure(figsize=(6,3.5),dpi=300)
ax = plt.subplot(111)
for alg in algorder:
    cur_df = df[df["r_algorithm"] == alg]
    ax.semilogy(
        cur_df["p"]/100,
        cur_df["a_throughput"],
        marker=markers[alg],
        label=alg,
    )
xs = np.arange(0,1.1,.1)
cur_df = df[df["r_algorithm"] == "simple"]
ax.semilogy(
    xs,
    np.repeat(cur_df["a_throughput"].mean(), len(xs)),
    label="simple",
    ls="--",
    marker=markers["simple"]
)

ax.semilogy(
    xs,
    np.repeat(11.3, len(xs)),
    label="sklearn",
    ls="-.",
    marker=markers["sklearn"]
)

ax.set_ylim(1,10**5)
ax.set_xlabel("Quantile Threshold")
ax.set_ylabel("Query Throughput")
ax.set_title("Adjusting Threshold: tmy3, n={}, d={}".format(
        human_format(df.num_train.iloc[0]),
        df.dim.iloc[0]))
ax.legend(loc=0)
ax.grid(True, ls="dotted", alpha=0.5)
fig.tight_layout()
fig.savefig("p_scale.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
